In [1]:
#run only once!
%load_ext autoreload
%autoreload 2
import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [2]:
import numpy as np
from phidl import LayerSet
from phidl import quickplot as qp
from phidl import Path, CrossSection, Device
import phidl.path as pp
import phidl.geometry as pg
import phidl
import matplotlib.pyplot as plt
from Schuster_Resonators.resonator import SchusterResonatorSmooth
from Schuster_Resonators.approx_Schuster import *
from CPW.cpw import *

In [3]:
def calibration_resonator(CapacitorHorizontalLength, 
                            CapacitorVerticalLength, 
                            CapacitorWidth,
                            NumberOfBends, 
                            InductorVerticalLength,
                            InductorHorizontalLength,
                            InductorWidth,
                            InductorEndLength,
                            TaperWidth,
                            TaperLength,
                            SpacingC0, 
                            SpacingCc,
                            buffer_box):
    
    #Layers
    ls = LayerSet()
    ls.add_layer('Etch', gds_layer=0, color = 'red')
    ls.add_layer('Cap', gds_layer=1, color = 'blue')
    ls.add_layer('Ind', gds_layer=2, color = 'grey')
    ls.add_layer('Box', gds_layer=3, color = 'green')


    #Shapes
    Resonator, Etch, Dcap, Dind, Droute = SchusterResonatorSmooth(CapacitorHorizontalLength, 
                                                  CapacitorVerticalLength, 
                                                  CapacitorWidth,
                                                  NumberOfBends, 
                                                  InductorVerticalLength,
                                                  InductorHorizontalLength,
                                                  InductorWidth,
                                                  InductorEndLength,
                                                  TaperWidth,
                                                  TaperLength,
                                                  SpacingC0, 
                                                  SpacingCc,
                                                  calib = True)
    
    #Final devices
    FinalDevice = Device('FinalDevice')
    FinalDevice.add_polygon(Dcap.get_polygons(), layer = ls['Cap'])
    FinalDevice.add_polygon(Dind.get_polygons(), layer = ls['Ind'])
    FinalDevice.add_polygon(Droute.get_polygons(), layer = ls['Cap'])
    FinalDevice.add_polygon(Etch.get_polygons(), layer = ls['Etch'])
    # box_poly = np.array(Etch.get_polygons()[0])
    # for i in range(box_poly.shape[0]):
    #     for j in range(box_poly.shape[1]):
    #         box_poly[i][j] += (box_poly[i][j]/np.abs(box_poly[i][j]))*buffer_box

    # box = pg.Polygon(box_poly, gds_layer=ls['Box'],gds_datatype= 0, parent = None )
    
    # FinalDevice.add_polygon(box, layer = ls['Box'])
    return FinalDevice

In [5]:
# ls = LayerSet()
# ls.add_layer('Ground', gds_layer=0, color = 'red')
# ls.add_layer('Metal', gds_layer=1, color = 'blue')

 
CapacitorHorizontalLength = 300
CapacitorVerticalLength = 200
CapacitorWidth = 20

NumberOfBends= 10
InductorVerticalLength = 15
InductorHorizontalLength = 150
InductorEndLength = 10
InductorTotalLength= InductorVerticalLength*(NumberOfBends+4) + InductorEndLength +InductorHorizontalLength*NumberOfBends
InductorWidth= np.arange(0.1, 0.7, 0.1)
TaperWidth= 20

SpacingC0 = 5
SpacingCc = 10
TaperLength = 5

for w in InductorWidth:
    CalibrationDevice = calibration_resonator(CapacitorHorizontalLength, 
                                                CapacitorVerticalLength, 
                                                CapacitorWidth,
                                                NumberOfBends, 
                                                InductorVerticalLength,
                                                InductorHorizontalLength,
                                                w,
                                                InductorEndLength,
                                                TaperWidth,
                                                TaperLength,
                                                SpacingC0, 
                                                SpacingCc,
                                                50)

# qp(CalibrationDevice)
    CalibrationDevice.write_gds(r'C:\Users\LocalAdmin\Documents\GitHub\resonators\gds_files\20241125-SiRes-Al/calib-w0p' + str(int(w*10)) + '-v2')

# qp(CalibrationDevice)
